# Import

In [ ]:
import glob
import time
import joblib
import pickle
import datetime
import numpy as np
import pandas as pd
import lightgbm as lgb
from influxdb import InfluxDBClient
from joblib import Parallel, delayed
from sklearn.base import BaseEstimator, TransformerMixin
pd.options.mode.chained_assignment = None

# Variables

In [ ]:
file_location = '/data/liufengyuan'
f_index = ['ticker', 'tradeDate']
f_x_221 = pickle.load(open(file_location + '/pct1_cal/dailyprediction/f_x_221', 'rb'))
extra_factors = ['volume', 'cachgPct', 'thecommittee', 'askVolume1',
       'bidVolume1', 'caQrr', 'caTr', 'OCVP1',
       'Open/vwap-1', 'Gap']
_extra_factors = ['jhjj_FACTORS_' + x for x in extra_factors]
f_x = f_x_221.copy()
f_x.extend(_extra_factors)
print(len(f_x_221), len(f_x))

# Deal with Data

* Label

In [ ]:
# label
#pct1_label = pd.read_csv(file_location + '/raw_data/pct1_8_12_open.csv')
#pct1_label['pct1_open_rank'] = pct1_label.groupby('tradeDate')['pct1_open'].rank(pct = True)
#pct1_label.to_csv(file_location + '/labels/{}.csv'.format('pct1_open'), index=False)
pct1_label = pd.read_csv('/data/liufengyuan/labels/pct1_open.csv')
pct1_label

* 2017-01-03 - 2022-07-07 alter 221 + jhjj 10

In [ ]:
alter221_jhjj_data = pd.read_csv(file_location + '/pct1_cal/每日预测/221_jingjia_data.csv')
alter221_jhjj_data

* alter 221 20220427-20220804

In [ ]:
def each_factor(file_loc, name, date):
    alpha = pd.read_csv(file_loc + name + '.csv')
    if alpha.columns.values.tolist()[0] == 'date':
        alpha = alpha.rename(columns={"date": "tradeDate"})
    alpha = alpha.set_index(['tradeDate']).stack().reset_index().rename(columns={'level_1': 'ticker', 0: name})
    alpha['ticker'] = alpha['ticker'].astype(int)
    if date != '':
        alpha = alpha[alpha.tradeDate == date]
    alpha = alpha.set_index(['ticker', 'tradeDate'])
    alpha = alpha.sort_index(ascending=True)
    return alpha

# combine all factors
def combine_factors(file_loc, f_x, date, thresh):        
    parallel_obj = Parallel(n_jobs=-1)(delayed(each_factor)(file_loc, name, date) for name in f_x)
    result = pd.concat(parallel_obj, axis=1)
    result = result.reset_index()
    result = result.sort_values(by=['ticker', 'tradeDate']).reset_index(drop=True)
    result = result.dropna(thresh=thresh)
    return result

In [ ]:
#读取昨天的普通因子
old_factors = combine_factors(file_location + '/pct1_cal/每日预测/data/', f_x_221, '', 67)
old_factors = old_factors[old_factors.tradeDate >= '2022-07-08'].reset_index(drop=True)
old_factors

In [ ]:
def get_jingjia(filenames):
    result_list = []
    for name in filenames:
        #print(name)
        alpha = pd.read_csv(name)
        alpha = alpha.drop(columns='Unnamed: 0')
        alpha  = alpha.rename(columns={"dataDate": "tradeDate"})
        alpha['ticker'] = alpha['ticker'].astype(int)
        alpha = alpha.set_index(['ticker', 'tradeDate'])
        alpha = alpha.sort_index(ascending=True)
        result_list.append(alpha)
    result = pd.concat(result_list)
    result = result.reset_index()
    return result

In [ ]:
filenames = glob.glob(file_location + '/pct1_cal/每日预测/jhjj_data/*.csv')
jhjj_data = get_jingjia(filenames)
jhjj_data = jhjj_data.sort_values(by=['ticker', 'tradeDate']).reset_index(drop=True)
jhjj_data = jhjj_data[jhjj_data.tradeDate >= '2022-07-08']
jhjj_data

20220805-20220812

In [ ]:
select_date = '2022-08-04'
factors = get_factors(select_date)
factors = factors[factors.tradeDate <= '2022-08-10']
factors

In [ ]:
alter221_jhjj_data = alter221_jhjj_data[f_index + f_x_221 + extra_factors]
old_factors = old_factors.merge(jhjj_data, on=['ticker', 'tradeDate'], how='left')
old_factors = old_factors[f_index + f_x_221 + extra_factors]
factors = factors[f_index + f_x_221 + _extra_factors]

In [ ]:
temp = pd.concat([alter221_jhjj_data, old_factors], axis=0)
temp.rename({'volume':'jhjj_FACTORS_volume', 
                                          'cachgPct':'jhjj_FACTORS_cachgPct', 
                                          'thecommittee':'jhjj_FACTORS_thecommittee', 
                                          'askVolume1':'jhjj_FACTORS_askVolume1',
                                          'bidVolume1':'jhjj_FACTORS_bidVolume1', 
                                          'caQrr':'jhjj_FACTORS_caQrr', 
                                          'caTr':'jhjj_FACTORS_caTr', 
                                          'OCVP1':'jhjj_FACTORS_OCVP1',
                                          'Open/vwap-1':'jhjj_FACTORS_Open/vwap-1', 
                                          'Gap':'jhjj_FACTORS_Gap'}, axis = 1, inplace=True)
temp

In [ ]:
factors = pd.concat([temp, factors], axis=0)
factors = reduce_mem_usage(factors)
factors.to_csv(file_location + '/pct1_cal/dailyprediction/gengpanyuce/raw_alter221_jhjj_data_170103_220810.csv', index=False)

In [ ]:
dates = factors.tradeDate.sort_values().unique()

In [ ]:
print('PARALLEL')
start = time.time()
parallel_obj = Parallel(n_jobs=-1)(delayed(run_df)(f_x, date, factors) for date in dates)
print(f'耗时:{time.time() - start}')
all_data = pd.concat(parallel_obj)
print('sort values')
all_data = all_data.sort_values(by=['ticker', 'tradeDate']).reset_index(drop=True)
print('reduce memory')
all_data = reduce_mem_usage(all_data)
all_data.to_csv(file_location + '/pct1_cal/dailyprediction/gengpanyuce/ExcExm_alter221_jhjj_data_170103_220810.csv', index=False)

# Useful Functions

In [ ]:
def check_missing_values(origin, new):
    print('The number of factors of my MODEL: ', len(origin))
    count1 = 0
    for file in origin:
        if file not in new:
            print(file)
            count1 += 1
    print('缺失值', count1)

    print('The number of factors in my DATABASE for each day: ', len(new))
    count2 = 0
    for file in new:
        if file not in origin:
            print(file)
            count2 += 1
    print('多余值', count2)

    return (count1 == 0 and count2 == 0)

In [ ]:
def ticekrToStr(job_sum):
    job_sum = job_sum[job_sum.ticker<700000]
    job_sum.loc[job_sum.ticker<10,'temp']='00000'
    job_sum.loc[(job_sum.ticker<100)&(job_sum.ticker>=10),'temp']='0000'
    job_sum.loc[(job_sum.ticker<1000)&(job_sum.ticker>=100),'temp']='000'
    job_sum.loc[(job_sum.ticker<10000)&(job_sum.ticker>=1000),'temp']='00'
    job_sum.loc[job_sum.temp==job_sum.temp,'ticker'] = job_sum[job_sum.temp==job_sum.temp]['temp']+job_sum[job_sum.temp==job_sum.temp]['ticker'].astype(str)
    del job_sum['temp']
    job_sum['ticker'] = job_sum['ticker'].astype(str)
    return job_sum

In [ ]:
def reduce_mem_usage(df, verbose=True):
    start_mem = df.memory_usage().sum() / 1024 ** 2
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024 ** 2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
class ExcludeExtreme(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.df_median = None
        self.df_standard = None

    def fit(self, df_columnList):
        self.df_median = df_columnList.median()
        self.df_standard = df_columnList.apply(lambda x: x - self.df_median[x.name]).abs().median()
        return self

    def scaller(self, x):
        self.di_max = self.df_median[x.name] + 5 * self.df_standard[x.name]
        self.di_min = self.df_median[x.name] - 5 * self.df_standard[x.name]
        x = x.apply(lambda v: self.di_min if v < self.di_min else v)
        x = x.apply(lambda v: self.di_max if v > self.di_max else v)
        return x

    def transform(self, df_columnList):
        df_columnList = df_columnList.apply(self.scaller)
        return df_columnList

In [ ]:
def get_factors(select_date):
    
    table_name = 'all_Factors'
    client = InfluxDBClient(host="175.25.50.116", port=12086, username="xtech", password="xtech2022", database="factor")
    result = client.query("select * from /"+ table_name +"/ where time >= '{}' ".format(select_date))
    result = pd.DataFrame(list(result.get_points()))

    factors = result.drop(columns= 'time')
    factors = factors.rename(columns= {'code': 'ticker'})
    factors['ticker'] = factors['ticker'].astype(int)
    factors.insert(0, 'tradeDate', factors.pop('tradeDate'))
    factors.insert(0, 'ticker', factors.pop('ticker'))
    factors['tradeDate'] = factors.tradeDate.apply(lambda x: x[:-9])
    factors = factors.dropna(thresh=70)
    if 'ga_FACTORS_sfactor175 ' in factors.columns.to_list():
        print('Input factor name \'ga_FACTORS_sfactor175 \' should be modified.')
        factors = factors.rename(columns={'ga_FACTORS_sfactor175 ': 'ga_FACTORS_sfactor175'})
    else:
        factors['ga_FACTORS_sfactor175'] = np.NAN
    return factors

In [ ]:
def run_df(f_x_221, date, df):
    print('----- DATE {}------'.format(date))
    pd.options.mode.chained_assignment = None
    df2 = df[df.tradeDate == date]

    mevtransformer = ExcludeExtreme()
    mevtransformer.fit(df2[f_x_221])
    df2[f_x_221] = mevtransformer.transform(df2[f_x_221])

    df2 = df2[f_index + f_x_221]
    return df2

# Research

In [ ]:
df1 = all_data.merge(pct1_label, on=['ticker', 'tradeDate'], how='left')
df1

In [ ]:
label_list = ['pct1_open', 'pct1_open_rank']
f_y_list =  label_list # 'pct_dmean'
print(f_y_list)
#f_x = pickle.load(open("pct1_cal/f_x_066", "rb"))

#e: extra, m: filted
data_source = 'Alter221_extra'
this_file_location = file_location + '/pct1_cal/dailyprediction/gengpanyuce'
model_list = ['LinearRegression', 'RidgeR', 'DecisionTreeR', 'XGBoostR', 'LGBMRegressor']
model_name = model_list[4]
target_types = ['r', 'c'] # 分类问题还是回归问题 r 回归问题 c 分类问题
target_type = target_types[0]
####========需要修改的全局参数========####

In [ ]:
dates = df1.tradeDate.sort_values().unique()
epoch_ts = list(dates)
f_index = ['ticker', 'tradeDate']
#result_name = '{}_{}_{}_{}'.format(data_source, model_name, f_y, target_type)
#print(result_name)

update = 22 # 训练长度：22天
train_si = epoch_ts.index('2017-01-03') # included. '2017-01-03'
train_ei = epoch_ts.index('2019-01-02') # excluded. '2018-12-28'
test_si = epoch_ts.index('2019-01-02') # included. '2019-01-02'
test_ei = epoch_ts.index('2019-02-01') # excluded. '2019-01-31' '2019-02-01'
test_fi = len(epoch_ts) - 1 # excluded. '2019-01-16'

# number of epochs，循环次数
num_epoch = round((test_fi - test_ei) / update)
epoch_range = range(0, num_epoch + 1)
timediff = pd.Timedelta(100,unit='d')

In [ ]:
def load_lrlist(lr_max,lr_min,num_rounds):
    lrlist = [lr_max+(lr_min-lr_max)*(np.log(i)/np.log(num_rounds)) for i in range(1,num_rounds+1)]
    return lrlist

In [ ]:
lr_max = 0.05
lr_min = 0.02
n_jobs = 20

In [ ]:
#gbm params
params =  {
    'boosting_type' : 'gbdt',
    'num_leaves' : 31,
    'feature_fraction' : 0.8,
    'bagging_fraction' : 0.8,
    'lambda_l1':1,
    'lambda_l2':10,
    'max_bin' : 64,
    'num_boost_round': 200,
    'learning_rate' : 0.04,
    'min_data_in_leaf':10,
    'num_threads': n_jobs
}
params['objective'] = 'regression'
params['metric'] = {'l2', 'auc'}
lrlist = load_lrlist(lr_max,lr_min,params['num_boost_round'])
params['lrlist'] = lrlist

In [ ]:
for f_y in f_y_list:
    result_name = '{}_{}_{}_{}'.format(data_source, model_name, f_y, target_type)
    print(result_name)

    df_result_all = pd.DataFrame()
    total_IC = 0
    start = time.time()
    #epoch_range = range(0, 1)

    for epoch in epoch_range:
        #epoch = num_epoch + 1
        print('----- EPOCH {}------'.format(epoch))
        update_n = epoch * update
        # get a list of train dates
        epoch_t_train = epoch_ts[train_si + update_n : train_ei + update_n]
        # get a list of test dates
        epoch_t_test = epoch_ts[test_si + update_n : test_ei + update_n]
        df_train = df1[df1.tradeDate.apply(lambda x: x in epoch_t_train)].reset_index(drop=True)
        df_test = df1[df1.tradeDate.apply(lambda x: x in epoch_t_test)].reset_index(drop=True)
        print('预测时间：', epoch_t_test)
        print('数据大小：', df_train.shape, df_test.shape)

        # 数据筛选 删除上市100天以内的
        #a = pd.to_datetime(df_train.tradeDate)
        #b = pd.to_datetime(df_train.listData)
        #df_train = df_train[a-b > timediff]
        #a = pd.to_datetime(df_test.tradeDate)
        #b = pd.to_datetime(df_test.listData)
        #df_test = df_test[a-b > timediff]

        # 获得 x
        x_train = df_train[f_x].values
        x_test = df_test[f_x].values
        print('处理后x:', x_train.shape, x_test.shape)

        # 获得y
        y_train = df_train[f_y].values
        y_test = df_test[f_y].values
        #y_train = assign_weight(y_train, weightdic)
        #y_test= assign_weight(y_test, weightdic)
        #y_train['weight'] = 1
        #y_test['weight'] = 1
        print('处理后y:', y_train.shape, y_test.shape)
        #model = XGBRegressor(n_estimators=863, max_depth=6, learning_rate = 0.0142, subsample = 0.68)
        #model = lgb.LGBMRegressor(max_depth=5, num_leaves=10, learning_rate=0.09, n_estimators=100)
        #model = lgb.LGBMRegressor(max_depth=6, num_leaves=25, learning_rate=0.1, n_estimators=100)
        # {'n_estimators': 100, 'learning_rate': 0.01, 'subsample': 0.7, 'max_depth': 5, 'subsample_freq': 5, 'reg_lambda': 0, 'num_leaves': 10, 'min_child_samples': 80, 'max_bin': 20}
        #model = lgb.LGBMRegressor(num_leaves = 15, min_child_samples=200, max_bin = 256, n_estimators=100, \
        #    importance_type='gain', deterministic= True, n_jobs=-1, subsample=0.9, subsample_freq=2)
        # 'n_estimators': 100, 'learning_rate': 0.07519120996144206, 'subsample': 0.7489953327400968, 'max_depth': 6, 'subsample_freq': 8, 'reg_lambda': 50, 'num_leaves': 60
        #model = lgb.LGBMRegressor(num_leaves= 60, n_estimators=100, learning_rate= 0.07519, reg_lambda = 50,
        #subsample= 0.7489953, subsample_freq=8, max_depth=6)
        #model = lgb.LGBMRegressor(**params)
        #model.fit(x_train, y_train, eval_set=[(x_train,y_train), 
        #(x_test,y_test)], eval_metric='l2')
        lgb_train = lgb.Dataset(x_train, y_train)
        model = lgb.train(params,
                        lgb_train,
                        num_boost_round=params['num_boost_round'],
                        callbacks = [lgb.reset_parameter(learning_rate = params['lrlist'])]
                        )
        y_pred = model.predict(x_test)

        # 获得结果
        print(f'耗时:{time.time() - start}')
        print('get result')
        df_result = df_test[f_index].copy()
        df_result['y'] = y_test
        df_result['y_pred'] = y_pred
        IC = df_result[['y', 'y_pred']].corr().iloc[0,1]
        print(IC)
        df_result_all = df_result_all.append(df_result)

    print(f'耗时:{time.time() - start}') 
    print('sort values')
    df_result_all = df_result_all.sort_values(by=['ticker', 'tradeDate']).reset_index(drop=True)
    IC = df_result_all[['y', 'y_pred']].corr().iloc[0,1]
    today = (datetime.datetime.now()).strftime("%Y-%m-%d:%H:%M:%S")
    df_result_all.to_csv('{}/{}{}_{}.csv'.format(this_file_location, result_name, today, round(IC, 4)), index=False)
    print('======== COMPLETED {} {} ========'.format(model_name, IC))


# Get Result

In [ ]:
def convertFile(filenames, file_location):
    for file in filenames:
        name = file[:-30] + file[-11:-4]
        print(name)
        df = pd.read_csv(file_location + file)
        df.rename(columns = {'tradeDate': 'date', 'y_pred': 'prediction'}, inplace = True)
        df['date'] = df.date.apply(lambda x: str(x).replace('-', ''))
        df = df[['ticker', 'date', 'prediction']]

        dates = df.date.sort_values().unique()
        def run(date):
            print(date)
            temp = df[df.date == date]
            temp = temp.sort_values(by=['prediction'], ascending= False).reset_index(drop=True)
            #temp.to_csv('Predictions/zz500_{}/{}.csv'.format(name, date), index=False)
            temp.to_csv('{}{}/{}.csv'.format(file_location, name, date), index=False)

        print('PARALLEL')
        start = time.time()
        parallel_obj = Parallel(n_jobs=48)(delayed(run)(date) for date in dates)
        print(f'耗时:{time.time() - start}')

In [ ]:
filenames = ['Alter221_extra_LGBMRegressor_pct1_open_r2022-08-16:11:15:55_0.1283.csv',
'Alter221_extra_LGBMRegressor_pct1_open_rank_r2022-08-16:11:50:58_0.1707.csv']
convertFile(filenames, this_file_location + '/')